#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 2:  Link Analysis -- HITS + SEO

### 100 points [5% of your final grade]

### Due: Sunday, February 25, 2018 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test HITS algortihm over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Wednesday, February 28, 2018 at 11:59pm.

# Part 1: HITS (70 points)

## A re-Tweet Graph

In this assignment, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *HITS.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are weighted and directed. If Bob retweets Alice's tweets 10 times, there is an edge from Bob to Alice with weight 10, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the HITS algorithm on the graph you build here.


In [1]:
# your code here 
import json
import numpy as np
from collections import defaultdict
from collections import Counter, OrderedDict
data = []
json_objects = []
edge_list = []
with open('HITS.json','r') as json_data:
    for line in json_data:
        data.append(line)
#Generating edge list
for json_string in data:
    userID = (json.loads(json_string))["user"]["id"]
    sourceID = (json.loads(json_string))["retweeted_status"]["user"]["id"]
    if userID != sourceID:
        edge_list.append((userID,sourceID))
temp_list = []
# print len(edge_list)
#Adding weights to edges
for edge,count in Counter(edge_list).items():
    if count > 1:
        edge = edge + (count,)
    else:
        edge =  edge + (1,)
    temp_list.append(edge)
edge_list = temp_list
#Generating graph from edge list
def create_graph(edge_list):
    graph = defaultdict()
    for edge in edge_list:
        neighbour_list = []
        node = edge[0]
        for edge in edge_list:
            if edge[0] == node:
                neighbour_list.append((edge[1],edge[2]))
        graph[node] = sorted(neighbour_list)
    return graph    
graph = create_graph(edge_list)
graph = OrderedDict(sorted(graph.items()))

We will not check the correctness of your graph. However, this will affect the HITS results later.

## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

In [2]:
# your code here
def create_adjacency_matrix(edge_list,nodes,weighted_flag = True):
    adjacency_matrix = []
    temp_list = [(x,y) for (x,y,z) in edge_list]
    for source in nodes:
        weight_list = []
        for dest in nodes:
            if (source,dest) not in temp_list:
                weight_list.append(0)
            else:
                if weighted_flag:
                    weight_list.append(edge_list[temp_list.index((source,dest))][2])
                else:
                    weight_list.append(1)
        adjacency_matrix.append(weight_list)
    return np.matrix(adjacency_matrix)
weighted_adj_matrix = create_adjacency_matrix(edge_list,graph.keys())
unweighted_adj_matrix = create_adjacency_matrix(edge_list,graph.keys(),False)

        

In [14]:
#Create initial score vectors
import itertools
hub_scores = np.array([1 for index in range(weighted_adj_matrix.shape[0])])
hub_scores = hub_scores[:, np.newaxis]
auth_scores = np.array([1 for index in range(weighted_adj_matrix.shape[0])])
auth_scores = auth_scores[:, np.newaxis]
def hits_algorithm(adj_matrix,iterations):
    global hub_scores,auth_scores
    adj_matrix_transpose =  adj_matrix.transpose()
    l = 1
    while (l <= iterations):    
        auth_scores = np.dot(adj_matrix_transpose,hub_scores)
        hub_scores = np.dot(adj_matrix,auth_scores)
        auth_scores = np.divide(auth_scores,np.sqrt(np.sum(np.square(auth_scores))))
        hub_scores = np.divide(hub_scores,np.sqrt(np.sum(np.square(hub_scores))))
        l=l+1
    return auth_scores,hub_scores
def print_top_hubs_authorities(auth_scores,hub_scores):
    auth_scores = list(itertools.chain.from_iterable(auth_scores.tolist()))
    hub_scores = list(itertools.chain.from_iterable(hub_scores.tolist()))
    sorted_auth = sorted(auth_scores, reverse = True)
    sorted_hub = sorted(hub_scores, reverse = True)
    print "Top 10 Hub Scores"
    print "Rank  UserID        Hub_Score"
    for i in range(10):
        print str(i+1)+"   "+str(graph.keys()[hub_scores.index(sorted_hub[i])])+"-"+str(sorted_hub[i])
    print "\nTop 10 Authority Scores"
    print "Rank  UserID     Authority_Score"
    for i in range(10):
        print str(i+1)+"   "+str(graph.keys()[auth_scores.index(sorted_auth[i])])+"-"+str(sorted_auth[i])
a_scores, h_scores = hits_algorithm(weighted_adj_matrix,5)
print "***************Weighted Adjacent Matrix***************\n"
print_top_hubs_authorities(a_scores,h_scores)
a_scores, h_scores = hits_algorithm(unweighted_adj_matrix,5)
print "\n************Unweighted Adjacent Matrix***************\n"
print_top_hubs_authorities(a_scores,h_scores)

***************Weighted Adjacent Matrix***************

Top 10 Hub Scores
Rank  UserID        Hub_Score
1   3068706044-0.60446316755
2   3093940760-0.292355614296
3   2194518394-0.243196988131
4   2862783698-0.198986122992
5   3029724797-0.166964010244
6   3092183276-0.165700124731
7   3001500121-0.164751979453
8   2990704188-0.150541937005
9   3086921438-0.126879329452
10   3021183212-0.119216412242

Top 10 Authority Scores
Rank  UserID     Authority_Score
1   3042570996-0.533629562007
2   3065514742-0.472074824932
3   1638625987-0.431519114931
4   3077733683-0.278121251172
5   3039321886-0.223996645179
6   1358345766-0.174870221902
7   3154266823-0.128032530459
8   3077695572-0.117479630125
9   3019659587-0.113449690353
10   610166901-0.0997414556842

************Unweighted Adjacent Matrix***************

Top 10 Hub Scores
Rank  UserID        Hub_Score
1   2990704188-0.229348744293
2   3001500121-0.17129564978
3   3158881135-0.138021047969
4   3068706044-0.137444976445
5   2797247939

# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **kbeznak parmatonic** --- two terms, lower case, no quote. As of today (Feb 16, 2018), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization#White_hat_versus_black_hat_techniques).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

https://saa14.github.io/kbeznak-parmatonic/

Used Github Pages as a host for my Jekyll-Based Website
(URL for indexed images - https://drive.google.com/open?id=1b_xxsY5q0eFnJlXCN5lmO6HbSAJwHh6N)

What's your strategy? (2-4 paragraphs)


# Answer
## SEO Strategies Used

### Getting Indexed Quickly

- Submitted the website URL to Search Engines - Used Google Webmaster Tools and Bing Webmaster Tools 
- Installed Google Analytics using Jekyll plugin
- Created and submitted website's sitemap to Google Search Console and  Bing Webmaster Tools (used the Jekyll plugin to create sitemap.xml) 

### On-Page SEO
This is used to improve the relevancy of the website and help the search engines to determine what the site is really about.
A structured site is more likely to be indexed higher. Concise anchor texts and simple navigation also helps. 

- Used "kbeznak parmatonic" in the title 
- Used the phrase "Official Homepage" in the title
- Used "kbeznak parmatonic" in the heading tags(h1,h2,h3,h4)  as much as possible
- Created images with the words "kbeznak parmatonic" and used "kbeznak parmatonic" as the alt text for it
- Used "kbeznak parmatonic" in the meta description, author and keywords
- Populated the page with AI generated poetry for original content (Used [Poetry generator](https://www.poetrygenerator.ninja/poem/b494f542e8f2a8c2),[Inspirobot](http://inspirobot.me/) ) 
- Used dofollow links on the homepage to other pages

### Off-Page SEO
This is used to build up high quality links (on other platforms) to the site and thus increase its rank. Backlinks from popular sites help in raising the rank for the website.
- Social Sites 

Created profiles and pages for "kbeznak parmatonic" on the following platforms

[Facebook Page](https://www.facebook.com/kbeznakparmatonicpoems/)

[Instagram Page](https://www.instagram.com/kbeznakparmatonicsaa14/)

[Reddit Profile](https://www.reddit.com/user/kbeznakparmatonic_)

[Twitter Profile](https://twitter.com/kbeznak_p)

[Stack Exchange Profile](https://stackexchange.com/users/13029136/kbeznak-parmatonic)

[Tumblr Page](https://www.tumblr.com/blog/kbeznakparmatonicposts)

Also linked these pages on the website.

- Posts on other platforms

Created reddit posts (in subreddits for random items) sharing the website, youtube video with the website in description and tweets sharing the website.

- Commented on posts about this topic on Quora, Reddit, StackExchange (sharing the website)

### Future SEO ideas
- Adding more original content
- Adding more content sharing the website on popular platforms
- Guestposts on other blogs on this topic (Wordpresss, Blogger)
- Getting shared by more autoritative sources
- Links with related sites on this topic